In [2]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# --- STEP 1: Data Preparation ---
# Ensure no NaNs exist (Critical Fix)
df = pd.read_csv('/content/reddit_preprocessed.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.dropna()
df['clean_comment'] = df['clean_comment'].astype(str)

print(f"Data ready for Exp 3. Total rows: {len(df)}")

# --- STEP 2: Define the Range of max_features ---
# We test a range of values. 'None' means use all features.
feature_ranges = [500, 1000, 3000, 5000, 10000, 20000, None]

# Vectorizer type with bi-gram setting (based on Exp 2 results)
VectorizerClass = TfidfVectorizer

# Set the experiment name
mlflow.set_experiment("EXP 3 - Max Features Tuning")

print(f"\n🧪 Starting Experiment 3 with {VectorizerClass.__name__}...\n")

# --- STEP 3: Experiment Loop ---
for max_feat in feature_ranges:
    # Handle the 'None' case for naming
    feat_name = str(max_feat) if max_feat is not None else "All"
    run_name = f"MaxFeat_{feat_name}"

    print(f"🚀 Running: {run_name} ...")

    with mlflow.start_run(run_name=run_name):
        # 1. Log Parameters
        mlflow.log_param("max_features", feat_name)
        mlflow.log_param("vectorizer", VectorizerClass.__name__)
        mlflow.log_param("ngram_range", "(1, 1)") # Keeping n-gram constant

        # 2. Split Data
        # Using the same random_state ensures fair comparison
        X_train, X_test, y_train, y_test = train_test_split(
            df.clean_comment,
            df.category,
            test_size=0.2,
            random_state=42
        )

        # 3. Vectorize
        # We initialize the vectorizer with the current max_features value
        vectorizer = VectorizerClass(max_features=max_feat, ngram_range=(1, 2))

        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)

        # Log the actual number of features used (useful if max_features=None)
        actual_features = X_train_vec.shape[1]
        mlflow.log_metric("actual_feature_count", actual_features)

        # 4. Train Model
        rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
        rf.fit(X_train_vec, y_train)

        # 5. Evaluate
        y_pred = rf.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)

        # 6. Log Metrics
        mlflow.log_metric("accuracy", accuracy)

        # Optional: Detailed report
        report = classification_report(y_test, y_pred, output_dict=True)
        mlflow.log_metric("f1_macro", report['macro avg']['f1-score'])

        print(f"   ✅ Features: {actual_features} | Accuracy: {accuracy:.4f}")

print("\n🎉 Experiment 3 Finished!")

Data ready for Exp 3. Total rows: 36662


2026/02/13 18:03:04 INFO mlflow.tracking.fluent: Experiment with name 'EXP 3 - Max Features Tuning' does not exist. Creating a new experiment.



🧪 Starting Experiment 3 with TfidfVectorizer...

🚀 Running: MaxFeat_500 ...
   ✅ Features: 500 | Accuracy: 0.6603
🏃 View run MaxFeat_500 at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3/runs/18dd1d39cb2341b1b750e9556de02523
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3
🚀 Running: MaxFeat_1000 ...
   ✅ Features: 1000 | Accuracy: 0.6588
🏃 View run MaxFeat_1000 at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3/runs/2619f8cc21f44c58b111020220846b1b
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3
🚀 Running: MaxFeat_3000 ...
   ✅ Features: 3000 | Accuracy: 0.6551
🏃 View run MaxFeat_3000 at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3/runs/085cc4e64ccb46aba0d37c458c4e610e
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddi

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


   ✅ Features: 449790 | Accuracy: 0.4375
🏃 View run MaxFeat_All at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3/runs/4f87e13cc6ba4fcf94abadfaf3519353
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/3

🎉 Experiment 3 Finished!


In [1]:
!pip install mlflow dagshub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.

In [1]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4657cbbf-f6ee-4bd0-baa3-194af90329c8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6ee7f1c470714e392bbbe1a1a4ee362b590d129ce955dc92bfa17d1ffa6b5823




Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!